# Lesion classification

In [ ]:
import sys 
import os
sys.path.append(os.path.abspath("/net/people/plgamyshenin/masters-thesis---melanoma-analysis-with-fnn/experiment--2"))

from lib.auc import calculate_auc
from lib.isic_loader import isic_loader
from lib.fractal_percolation import FractalPercolation
from lib.gradcam import display_gradcam, make_gradcam_heatmap, get_img_array

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [ ]:
tf.get_logger().setLevel('ERROR')

### Model configuration

- Model architecture: Inception ResNet V2
- Fractal module used: with Percolation
- Number of layers tuned: 1

In [ ]:
DATA_SIZE = None
EPOCHS = 30


O_MODEL = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
            include_top=False,
            weights='imagenet',
            input_shape=(299, 299, 3),
            pooling='avg'
        )

F_MODEL = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
            include_top=False,
            weights='imagenet',
            input_shape=(299, 299, 3),
            pooling='avg'
        )

IMAGE_PREPROCESSING = tf.keras.applications.inception_resnet_v2.preprocess_input
FRACTAL_MODULE = FractalPercolation
UNFROZEN_LAYERS = ['conv7_b']
GRADCAM_LAYER = 'conv_7b'

### Load the ISIC Archive dataset

In [ ]:
class_weight, (training_set, validation_set, testing_set) = isic_loader(size=DATA_SIZE)

### Model definition and callbacks

In [ ]:
class LesionClassifier(tf.keras.Model):
    TARGET_WIDTH, TARGET_HEIGHT = 299, 299
    
    def __init__(self, class_number):
        super(LesionClassifier, self).__init__()
        
        self.fractal_module = FRACTAL_MODULE()
        self.resize = tf.keras.layers.Resizing(
            width=self.TARGET_WIDTH,
            height=self.TARGET_HEIGHT
        )
        self.rescale = tf.keras.layers.Rescaling(scale=1./255)
        
        self.o_model = O_MODEL
        for layer in self.o_model.layers:
            if layer.name in UNFROZEN_LAYERS:
                layer.trainable = True
            else:
                layer.trainable = False
                
        self.f_model = F_MODEL
        for layer in self.f_model.layers:
            if layer.name in UNFROZEN_LAYERS:
                layer.trainable = True
            else:
                layer.trainable = False
                
        self.fusion = tf.keras.layers.Concatenate()
        self.dense = tf.keras.layers.Dense(class_number, activation='softmax')
        
    def call(self, inputs):
        f_outputs = self.fractal_module(inputs)
        f_outputs = self.resize(f_outputs)
        f_outputs = self.rescale(f_outputs)
        f_outputs = self.f_model(f_outputs)
        
        o_outputs = self.rescale(inputs)
        o_outputs = self.o_model(o_outputs)
        
        outputs = self.fusion([f_outputs, o_outputs])
        outputs = self.dense(outputs)
        
        return outputs

In [ ]:
early_stop_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    min_delta=0.01, 
    patience=5, 
    restore_best_weights=True
)

### Training

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
with strategy.scope():
    model = LesionClassifier(class_number=len(training_set.class_indices))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(
    training_set, 
    validation_data=validation_set, 
    epochs=EPOCHS,
    class_weight=class_weight,
    callbacks=[
        early_stop_callback
    ]
)

### Testing

In [ ]:
groundtruth_labels = np.concatenate([testing_set[i][1] for i in range(len(testing_set))], axis=0)

In [ ]:
predicted_labels = model.predict(testing_set)

In [ ]:
fpr, tpr, auc_metric = calculate_auc(testing_set, groundtruth_labels, predicted_labels)

In [ ]:
auc_metric

In [ ]:
%matplotlib inline 

plt.rcParams['axes.titley'] = -0.3

for diagnosis in testing_set.class_indices:
    plt.plot(fpr[diagnosis], tpr[diagnosis], label=diagnosis)
    
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(bbox_to_anchor=(1.6, 1), loc="upper right")
plt.show()

### GradCam

In [ ]:
melanoma_im_path = f"{os.environ['SCRATCH']}/isic-archive/ISIC_0000031.jpg"

In [ ]:
display(Image(melanoma_im_path, width=600))

In [ ]:
img_array = IMAGE_PREPROCESSING(get_img_array(melanoma_im_path, size=(299, 299)))

In [ ]:
heatmap = make_gradcam_heatmap(img_array, model.o_model, GRADCAM_LAYER)

In [ ]:
display_gradcam(melanoma_im_path, heatmap)